In [2]:
!pip install geopy

In [4]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np 

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [4]:
soup = BeautifulSoup(data, 'html.parser')
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [5]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [7]:
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens, Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


In [10]:
toronto_df_grouped.shape

(103, 3)

In [11]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


In [17]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [19]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [20]:
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [22]:
CLIENT_ID = 'E1GHDNTGVO3PHIZBDK5SSYI24BQMEMORYMUXLH5HASGFBU5F' # your Foursquare ID
CLIENT_SECRET = '21JMBEZEAFY0KARX03EE235D1FGCB2TBI3SKHIIPFQXZXDDR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: E1GHDNTGVO3PHIZBDK5SSYI24BQMEMORYMUXLH5HASGFBU5F
CLIENT_SECRET:21JMBEZEAFY0KARX03EE235D1FGCB2TBI3SKHIIPFQXZXDDR


In [23]:
radius = 700
LIMIT = 150

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [24]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2528, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater


In [25]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,41,41,41,41,41,41
M4K,East Toronto,"The Danforth West, Riverdale",92,92,92,92,92,92
M4L,East Toronto,"The Beaches West, India Bazaar",50,50,50,50,50,50
M4M,East Toronto,Studio District,84,84,84,84,84,84
M4N,Central Toronto,Lawrence Park,6,6,6,6,6,6
M4P,Central Toronto,Davisville North,19,19,19,19,19,19
M4R,Central Toronto,North Toronto West,40,40,40,40,40,40
M4S,Central Toronto,Davisville,62,62,62,62,62,62
M4T,Central Toronto,"Moore Park, Summerhill East",6,6,6,6,6,6


In [26]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 284 uniques categories.


In [27]:
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2528, 287)


,PostalCode,Borough,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game St

In [35]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).sum().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 287)


,PostalCode,Borough,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game St

In [29]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 23)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M4E,East Toronto,The Beaches,Pub,Gastropub,Bar,Health Food Store,Sandwich Place,Breakfast Spot,Café,Indian Restaurant,Indie Movie Theater,French Restaurant,Japanese Restaurant,Juice Bar,Mexican Restaurant,Greek Restaurant,Shoe Store,Tea Room,Electronics Store,Neighborhood,Ramen Restaurant,Coffee Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Pub,Café,Ice Cream Shop,Italian Restaurant,Fast Food Restaurant,Grocery Store,Bakery,Furniture / Home Store,Pizza Place,Restaurant,Bookstore,Yoga Studio,Gym,Convenience Store,Burger Joint,Business Service,Sandwich Place,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",Indian Restaurant,Grocery Store,Sandwich Place,Pet Store,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Discount Store,Café,Gym,Park,Burrito Place,Sushi Restaurant,Food & Drink Shop,Liquor Store,Pizza Place,Steakhouse,Fish & Chips Shop,Diner,Pub
3,M4M,East Toronto,Studio District,Café,Sandwich Place,Coffee Shop,American Restaurant,Bakery,Bar,Diner,Italian Restaurant,Restaurant,Gastropub,Latin American Restaurant,Brewery,Vietnamese Restaurant,Liquor Store,Lounge,Seafood Restaurant,Boutique,Bookstore,Market,Fish Market
4,M4N,Central Toronto,Lawrence Park,Photography Studio,Bus Line,Swim School,Park,Business Service,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio,Dive Bar,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Dog Run,Discount Store
5,M4P,Central Toronto,Davisville North,Park,Indian Restaurant,Ice Cream Shop,Sushi Restaurant,Food & Drink Shop,Café,Taco Place,Dog Run,Bar,Sandwich Place,Pharmacy,Greek Restaurant,Hotel,Gym,Breakfast Spot,Brewery,Pizza Place,Dessert Shop,Dumpling Restaurant,Donut Shop
6,M4R,Central Toronto,North Toronto West,Coffee Shop,Clothing Store,Café,Sporting Goods Shop,Italian Restaurant,Diner,Dessert Shop,Restaurant,Ramen Restaurant,Spa,Gift Shop,Salon / Barbershop,Chinese Restaurant,Rental Car Location,Flower Shop,Fast Food Restaurant,Gym Pool,Bakery,Pet Store,Park
7,M4S,Central Toronto,Davisville,Pizza Place,Sandwich Place,Coffee Shop,Café,Gym,Italian Restaurant,Dessert Shop,Restaurant,Fast Food Restaurant,Sushi Restaurant,Indian Restaurant,Greek Restaurant,Bar,Spa,Farmers Market,Brewery,Bubble Tea Shop,Ramen Restaurant,Pub,Japanese Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",Park,Thai Restaurant,Playground,Gym,Grocery Store,Dumpling Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Eastern European Restaurant,Discount Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Electronics Store
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Italian Restaurant,Restaurant,Sushi Restaurant,Bagel Shop,Light Rail Station,Pizza Place,Pub,Sandwich Place,Café,Skating Rink,Gym,Yoga Studio,Gastropub,Cantonese Restaurant,Liquor Store,Burger Joint,Chinese Restaurant,Chiropractor,Modern European Restaurant


In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 2, 0, 0, 0, 3, 0])

In [31]:
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged# check the last columns!

(39, 26)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Gastropub,Bar,Health Food Store,Sandwich Place,Breakfast Spot,Café,Indian Restaurant,Indie Movie Theater,French Restaurant,Japanese Restaurant,Juice Bar,Mexican Restaurant,Greek Restaurant,Shoe Store,Tea Room,Electronics Store,Neighborhood,Ramen Restaurant,Coffee Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Pub,Café,Ice Cream Shop,Italian Restaurant,Fast Food Restaurant,Grocery Store,Bakery,Furniture / Home Store,Pizza Place,Restaurant,Bookstore,Yoga Studio,Gym,Convenience Store,Burger Joint,Business Service,Sandwich Place,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Indian Restaurant,Grocery Store,Sandwich Place,Pet Store,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Discount Store,Café,Gym,Park,Burrito Place,Sushi Restaurant,Food & Drink Shop,Liquor Store,Pizza Place,Steakhouse,Fish & Chips Shop,Diner,Pub
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Sandwich Place,Coffee Shop,American Restaurant,Bakery,Bar,Diner,Italian Restaurant,Restaurant,Gastropub,Latin American Restaurant,Brewery,Vietnamese Restaurant,Liquor Store,Lounge,Seafood Restaurant,Boutique,Bookstore,Market,Fish Market
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Photography Studio,Bus Line,Swim School,Park,Business Service,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio,Dive Bar,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Dog Run,Discount Store
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Park,Indian Restaurant,Ice Cream Shop,Sushi Restaurant,Food & Drink Shop,Café,Taco Place,Dog Run,Bar,Sandwich Place,Pharmacy,Greek Restaurant,Hotel,Gym,Breakfast Spot,Brewery,Pizza Place,Dessert Shop,Dumpling Restaurant,Donut Shop
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Coffee Shop,Clothing Store,Café,Sporting Goods Shop,Italian Restaurant,Diner,Dessert Shop,Restaurant,Ramen Restaurant,Spa,Gift Shop,Salon / Barbershop,Chinese Restaurant,Rental Car Location,Flower Shop,Fast Food Restaurant,Gym Pool,Bakery,Pet Store,Park
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Pizza Place,Sandwich Place,Coffee Shop,Café,Gym,Italian Restaurant,Dessert Shop,Restaurant,Fast Food Restaurant,Sushi Restaurant,Indian Restaurant,Greek Restaurant,Bar,Spa,Farmers Market,Brewery,Bubble Tea Shop,Ramen Restaurant,Pub,Japanese Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Park,Thai Restaurant,Playground,Gym,Grocery Store,Dumpling Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Eastern European Restaurant,Discount Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Electronics Store
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,0,Coffee Shop,Italian Restaurant,Restaurant,Sushi Restaurant,Bagel Shop,Light Rail Station,Pizza Place,Pub,Sandwich Place,Café,Skating Rink,Gym,Yoga Studio,Gastropub,Cantonese Restaurant,Liquor Store,Burger Joint,Chinese Restaurant,Chiropractor,Modern European Restaurant


In [32]:
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 26)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Gastropub,Bar,Health Food Store,Sandwich Place,Breakfast Spot,Café,Indian Restaurant,Indie Movie Theater,French Restaurant,Japanese Restaurant,Juice Bar,Mexican Restaurant,Greek Restaurant,Shoe Store,Tea Room,Electronics Store,Neighborhood,Ramen Restaurant,Coffee Shop
20,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Bar,Gastropub,American Restaurant,Seafood Restaurant,Japanese Restaurant,Concert Hall,Thai Restaurant,Deli / Bodega,Salad Place,Lounge,Pizza Place,Beer Bar,Vegetarian / Vegan Restaurant,Gym,Bakery
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Café,Restaurant,Hotel,Concert Hall,Seafood Restaurant,Gastropub,Japanese Restaurant,Italian Restaurant,American Restaurant,Bar,Cocktail Bar,Deli / Bodega,Thai Restaurant,Bakery,Vegetarian / Vegan Restaurant,Gym,Beer Bar,Salad Place,Pub
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Sandwich Place,Café,Coffee Shop,Park,Pub,Pizza Place,Vegetarian / Vegan Restaurant,Burger Joint,Diner,American Restaurant,Indian Restaurant,Pharmacy,Grocery Store,Falafel Restaurant,Sports Bar,Latin American Restaurant,Liquor Store,Social Club,Steakhouse,Burrito Place
25,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,0,Coffee Shop,Café,Pub,Restaurant,Bookstore,Bakery,Pizza Place,Gym,Yoga Studio,Bar,Ice Cream Shop,Bistro,Electronics Store,Japanese Restaurant,Burrito Place,Thai Restaurant,Comic Shop,Park,Hotel,Bubble Tea Shop
26,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,0,Bar,Café,Chinese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Dessert Shop,Vietnamese Restaurant,Coffee Shop,Mexican Restaurant,Farmers Market,Burger Joint,Ramen Restaurant,Park,Comfort Food Restaurant,Tea Room,Dumpling Restaurant,Cocktail Bar,French Restaurant,Korean Restaurant,Massage Studio
27,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,0,Harbor / Marina,Airport Lounge,Sculpture Garden,Airport Terminal,Airport Service,Coffee Shop,Airport Gate,Airport Food Court,Airport,Boat or Ferry,Tunnel,Pier,Bar,Music Venue,Rental Car Location,Boutique,Plane,Dumpling Restaurant,Eastern European Restaurant,Falafel Restaurant
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Beer Bar,Seafood Restaurant,Gym,Italian Restaurant,Bakery,Hotel,BBQ Joint,Farmers Market,Cocktail Bar,Breakfast Spot,Sporting Goods Shop,Cheese Shop,Creperie,Gastropub,American Restaurant,Comfort Food Restaurant
29,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0,Coffee Shop,Café,Restaurant,Bar,Hotel,Italian Restaurant,Gastropub,Seafood Restaurant,Japanese Restaurant,Steakhouse,Sushi Restaurant,Tea Room,Bakery,Burger Joint,Beer Bar,Asian Restaurant,Deli / Bodega,Concert Hall,Thai Restaurant,Pizza Place
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Coffee Shop,Playground,Italian Restaurant,Storage Facility,Candy Store,Beer Store,Nightclub,Diner,Gas Station,Baby Store,Athletics & Sports,Bakery,Gym,Restaurant,Design Studio,Donut Shop,Farm


In [33]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,East Toronto,0,Pub,Gastropub,Bar,Health Food Store,Sandwich Place,Breakfast Spot,Café,Indian Restaurant,Indie Movie Theater,French Restaurant,Japanese Restaurant,Juice Bar,Mexican Restaurant,Greek Restaurant,Shoe Store,Tea Room,Electronics Store,Neighborhood,Ramen Restaurant,Coffee Shop
20,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Bar,Gastropub,American Restaurant,Seafood Restaurant,Japanese Restaurant,Concert Hall,Thai Restaurant,Deli / Bodega,Salad Place,Lounge,Pizza Place,Beer Bar,Vegetarian / Vegan Restaurant,Gym,Bakery
21,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Concert Hall,Seafood Restaurant,Gastropub,Japanese Restaurant,Italian Restaurant,American Restaurant,Bar,Cocktail Bar,Deli / Bodega,Thai Restaurant,Bakery,Vegetarian / Vegan Restaurant,Gym,Beer Bar,Salad Place,Pub
24,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Park,Pub,Pizza Place,Vegetarian / Vegan Restaurant,Burger Joint,Diner,American Restaurant,Indian Restaurant,Pharmacy,Grocery Store,Falafel Restaurant,Sports Bar,Latin American Restaurant,Liquor Store,Social Club,Steakhouse,Burrito Place
25,Downtown Toronto,0,Coffee Shop,Café,Pub,Restaurant,Bookstore,Bakery,Pizza Place,Gym,Yoga Studio,Bar,Ice Cream Shop,Bistro,Electronics Store,Japanese Restaurant,Burrito Place,Thai Restaurant,Comic Shop,Park,Hotel,Bubble Tea Shop
26,Downtown Toronto,0,Bar,Café,Chinese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Dessert Shop,Vietnamese Restaurant,Coffee Shop,Mexican Restaurant,Farmers Market,Burger Joint,Ramen Restaurant,Park,Comfort Food Restaurant,Tea Room,Dumpling Restaurant,Cocktail Bar,French Restaurant,Korean Restaurant,Massage Studio
27,Downtown Toronto,0,Harbor / Marina,Airport Lounge,Sculpture Garden,Airport Terminal,Airport Service,Coffee Shop,Airport Gate,Airport Food Court,Airport,Boat or Ferry,Tunnel,Pier,Bar,Music Venue,Rental Car Location,Boutique,Plane,Dumpling Restaurant,Eastern European Restaurant,Falafel Restaurant
28,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Beer Bar,Seafood Restaurant,Gym,Italian Restaurant,Bakery,Hotel,BBQ Joint,Farmers Market,Cocktail Bar,Breakfast Spot,Sporting Goods Shop,Cheese Shop,Creperie,Gastropub,American Restaurant,Comfort Food Restaurant
29,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Bar,Hotel,Italian Restaurant,Gastropub,Seafood Restaurant,Japanese Restaurant,Steakhouse,Sushi Restaurant,Tea Room,Bakery,Burger Joint,Beer Bar,Asian Restaurant,Deli / Bodega,Concert Hall,Thai Restaurant,Pizza Place
30,Downtown Toronto,0,Grocery Store,Café,Park,Coffee Shop,Playground,Italian Restaurant,Storage Facility,Candy Store,Beer Store,Nightclub,Diner,Gas Station,Baby Store,Athletics & Sports,Bakery,Gym,Restaurant,Design Studio,Donut Shop,Farm
